<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/Econophysics/Technical_Indicators_and_Trading_Decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from yfinance import download

# Download historical data for AAPL stock
df = download('AAPL', start='2010-01-01', end='2023-06-17')

# Calculate the Bollinger Bands
bb = BollingerBands(close=df['Close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

# Calculate the RSI
rsi = RSIIndicator(close=df['Close'], window=14)
df['rsi'] = rsi.rsi()

# Remove the first rows with missing indicator values
df = df.dropna()

# Define the target variable
df['target'] = df['Close'].shift(-1) > df['Close']
df['target'] = df['target'].map({True: 'Buy', False: 'Sell'})

# Remove the last row, which has a NaN target
df = df[:-1]

# Split the data
train_size = int(0.8 * df.shape[0])
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Check if either the training or testing set is empty
if train.empty or test.empty:
    raise ValueError("Insufficient data for training and testing.")

# Select the features
features = ['bb_bbm', 'bb_bbh', 'bb_bbl', 'rsi']
X_train = train[features]
y_train = train['target']
X_test = test[features]
y_test = test['target']

# Create and train the classifier
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X_train, y_train)

# Test the classifier
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

# Define the threshold for decision-making
threshold = 0.01  # Adjust as needed
num_days = 5  # Number of days to make the decision for

# Calculate the price change
df['price_change'] = df['Close'].shift(-1) - df['Close']

# Determine the target (1 for positive change, 0 for non-positive change)
df['target'] = np.where(df['price_change'] > 0, 1, 0)

# Map the target values to 'Buy' and 'Sell'
df['target'] = df['target'].map({1: 'Buy', 0: 'Sell'})

# Calculate the predicted price change
df['predicted_change'] = clf.predict(df[features])

# Shift the predicted_change column by the desired number of days
for i in range(1, num_days + 1):
    df[f'target_{i}'] = df['predicted_change'].shift(-i)

# Determine the decision for each day based on the threshold
for i in range(1, num_days + 1):
    df[f'decision_{i}'] = df[f'target_{i}'].apply(lambda x: 'Buy' if x == 'Buy' else ('Sell' if x == 'Sell' else 'Hold'))

# Print the decisions for each day
for i in range(1, num_days + 1):
    print(f"Decision for Day {i}: {df[f'decision_{i}'].iloc[0]}")

# Print the classification report for the first day
print(classification_report(df['target'], df['decision_1']))





Description:

The "Technical_Indicators_and_Trading_Decision.ipynb" code is a Jupyter Notebook that performs the following tasks:

Data Retrieval: It retrieves historical stock price data from Yahoo Finance for a specified ticker symbol.

Technical Indicator Calculation: It calculates various technical indicators, such as Relative Strength Index (RSI), Bollinger Bands, and Money Flow Index (MFI), using the Ta-Lib library and custom functions.

Data Preprocessing: It preprocesses the data by removing any rows with missing values and selecting the relevant features for training the machine learning model.

Machine Learning Model Training: It trains a Random Forest Classifier using the selected features and the target variable, which represents the predicted change in stock price.

Model Evaluation: It evaluates the performance of the trained model using precision, recall, and F1-score metrics. It also provides a classification report showing the performance for each class (Buy and Sell).

Trading Decision Generation: It generates trading decisions for a specified number of days into the future based on the predicted change in stock price. The decisions are mapped to 'Buy', 'Sell', or 'Hold' based on a predefined threshold.

Decision Evaluation: It evaluates the performance of the trading decisions using precision, recall, and F1-score metrics. It provides a classification report showing the performance for each class (Buy, Sell, and Hold).

The code aims to assist in generating trading decisions based on technical indicators and evaluating their performance.